In [15]:
from kaggle_environments import make
env = make('tictactoe', debug=True)

In [16]:
def verifica_vitoria(estado):
    # Verifica as linhas horizontalmente
    for i in range(0, 9, 3):
        if estado[i] == estado[i + 1] == estado[i + 2] and estado[i] != 0:
            return estado[i]
        
    # Verifica as colunas verticalmente
    for i in range(3):
        if estado[i] == estado[i + 3] == estado[i + 6] and estado[i] != 0:
            return estado[i]
        
    # Verifica a diagonal principal
    if estado[0] == estado[4] == estado[8] and estado[0] != 0:
        return estado[0]
    
    # Verifica a diagonal secundária
    if estado[2] == estado[4] == estado[6] and estado[2] != 0:
        return estado[2]
    
    # Se nenhuma das condições acima for atendida, retorna None (nenhum vencedor)
    return None

In [17]:
import math
import time

# Função utilitária para avaliar o estado atual do jogo
def utils(estado, jogador):
    vitoria = verifica_vitoria(estado)  # Verifica se houve vitória no estado atual
    tabuleiro = []  # Lista para armazenar o tabuleiro para análise
    pontuacao = 0  # Inicializa a pontuação como zero

    if vitoria:
        # Se houver vitória e o jogador atual for o vencedor, retorna 1
        if vitoria == jogador:
            return 1
        # Se houver vitória e o jogador atual não for o vencedor, retorna -1
        return -1
    else:
        # Gera possíveis linhas e colunas do tabuleiro para análise
        for i in range(0, 3):
            tabuleiro.append([estado[3*i], estado[1 + 3*i], estado[2 + 3*i]])
            tabuleiro.append([estado[i], estado[i + 3], estado[i + 6]])

            if i == 0:
                tabuleiro.append([estado[0], estado[4], estado[8]])
            if i == 2:
                tabuleiro.append([estado[2], estado[4], estado[6]])

        # Avalia as possíveis linhas e colunas do tabuleiro para pontuação
        for jogada in tabuleiro:
            count = 0
            if set(jogada) == {1, 2}:
                if jogada.count(muda_jogador(jogador)) == 2:
                    pontuacao += 0.005
            if not (set(jogada) == {1, 2} or set(jogada) == {0, 1, 2} or set(jogada) == {0}):
                if set(jogada) == {0, jogador}:
                    count = jogada.count(jogador)
                    pontuacao += 0.01 * count + (count - 1) * 0.02
                else:
                    count = jogada.count(muda_jogador(jogador))
                    pontuacao -= 0.01 * count + (count - 1) * 0.02

    return pontuacao  # Retorna a pontuação calculada


# Função para gerar os possíveis estados sucessores do tabuleiro
def sucessores(estado, jogador):
    sucessores = []

    for i in range(len(estado)):
        if estado[i] == 0:
            novo_estado = estado.copy()
            novo_estado[i] = jogador
            sucessores.append((novo_estado, i))

    return sucessores  # Retorna a lista de sucessores


# Função para alternar entre jogadores
def muda_jogador(jogador):
    return 1 if jogador == 2 else 2


# Algoritmo Max-Value do Minimax: retorna a pontuação máxima possível para o jogador atual
def max_value(estado, jogador, p, pos=None):
    pos_final = pos  # Armazena a posição final caso não haja mudanças
    if p == 0:
        return utils(estado, jogador), pos  # Se a profundidade for zero, retorna a pontuação do estado e a posição
    v = -math.inf  # Inicializa v com menos infinito para garantir que seja substituído por um valor maior

    for s, pos_linha in sucessores(estado, jogador):
        vitoria = verifica_vitoria(s)
        if vitoria == None:  # Se não houver vitória no estado sucessor
            if pos == None:  # Se não houver posição final definida
                # Calcula o score do estado sucessor usando o min_value (jogador adversário)
                if min_value(s, muda_jogador(jogador), p-1, pos_linha)[0] > v:
                    v = min_value(s, muda_jogador(jogador), p-1, pos_linha)[0]  # Atualiza v com o novo score
                    pos_final = pos_linha  # Atualiza a posição final com a nova posição
            else:
                v = max(v, min_value(s, muda_jogador(jogador), p-1, pos)[0])  # Atualiza v com o maior entre v e o score
        elif vitoria == jogador:  # Se o jogador atual vence no estado sucessor
            if pos:  # Se houver uma posição final definida
                return 1, pos  # Retorna 1 (vitória do jogador) e a posição final
            return 1, pos_linha  # Retorna 1 (vitória do jogador) e a nova posição

    return v, pos_final  # Retorna a maior pontuação e a posição correspondente


# Algoritmo Min-Value do Minimax: retorna a pontuação mínima possível para o jogador adversário
def min_value(estado, jogador, p, pos):
    if p == 0:
        return utils(estado, muda_jogador(jogador)), pos  # Se a profundidade for zero, retorna a pontuação do estado e a posição
    v = math.inf  # Inicializa v com infinito para garantir que seja substituído por um valor menor

    for s, pos_linha in sucessores(estado, jogador):
        vitoria = verifica_vitoria(s)
        if vitoria == None:  # Se não houver vitória no estado sucessor
            v = min(v, max_value(s, muda_jogador(jogador), p-1, pos)[0])  # Atualiza v com o menor entre v e o score do estado sucessor
        elif vitoria == jogador:  # Se o jogador atual vence no estado sucessor
            return -1, pos  # Retorna -1 (perda do jogador adversário) e a posição

    return v, pos  # Retorna a menor pontuação e a posição correspondente


# Função principal que chama o algoritmo Minimax
def minmax(tabuleiro, jogador):
    valor, pos = max_value(tabuleiro, jogador, 3)  # Executa o algoritmo Max-Value do Minimax com profundidade 3

    return pos  # Retorna a posição com a maior pontuação possível para o jogador atual


In [18]:
# Função para criar o agente de jogo
def my_agent(obs, reward):
    # Executa o algoritmo Minimax para determinar a próxima jogada do agente
    pos = minmax(obs['board'], obs.mark)
    return pos  # Retorna a posição escolhida pelo algoritmo Minimax

In [26]:
from kaggle_environments import evaluate

environment = 'tictactoe'
configuration = {}
steps = []
agents = ["random",my_agent]
num_episodes = 100

rewards = evaluate(environment, agents, configuration, steps, num_episodes)

In [27]:
print(rewards.count([1,-1]))
print(rewards.count([0,0]))
print(rewards.count([-1,1]))

3
1
95
